In [ ]:
import socket
import struct

UDP_IP   = "0.0.0.0"    # 本机监听所有网卡
UDP_PORT = 10088

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))

# 我们约定发送端是 77 个 double (8字节 each)，总共 616 字节
EXPECTED_LEN   = 77
EXPECTED_BYTES = EXPECTED_LEN * 8

# 采用 little-endian 格式解析: "<" 前缀 + "d" * 77
fmt = "<" + "d" * EXPECTED_LEN

print(f"Listening on {UDP_IP}:{UDP_PORT} ...")

while True:
    data, addr = sock.recvfrom(65535)
    if len(data) != EXPECTED_BYTES:
        print(f"[WARN] Received {len(data)} bytes, expected {EXPECTED_BYTES} -> skipping")
        continue

    # 解析二进制为 77 个 double
    values = struct.unpack(fmt, data)

    # 示例：取第 1 个和第 2 个值
    sim_time    = values[0]
    y_spcktime  = values[1]
    y_cb_vx     = values[2]
    # ... 其他字段都在 values[3], values[4], ... etc

    # 也可以直接打印全部
    print("Got 77 doubles from:", addr)
    print(values)
    print("----")
